# Cleaning and Preparing Data in Python - Museum of Modern Art Dataset

In [1]:
from csv import reader

## About the dataset

Dataset I will work with is the dataset with data about the art in the Museum of Modern Art (MoMA). MoMA, a museum in New York City, has one of the largest collections of modern art in the world.

Dataset consists of following columns:  
  
* Title: the title of the artwork,
* Artist: the name of the artist who created the artwork,
* Nationality: the nationality of the artist,
* BeginDate: the year in which the artist was born,
* EndDate: the year in which the artist died,
* Gender: the gender of the artist,
* Date: the date that the artwork was created,
* Department: the department inside MoMA to which the artwork belongs.

## Opening the dataset

In [2]:
with open('artworks.csv', encoding='UTF-8') as file:
    read_file = reader(file)
    moma = list(read_file)

In [3]:
moma_header = moma[1]
moma_header

['Dress MacLeod from Tartan Sets',
 'Sarah Charlesworth',
 '(American)',
 '(1947)',
 '(2013)',
 '(Female)',
 '1986',
 'Prints & Illustrated Books']

In [26]:
moma_data = moma[1:]


## Exploring the dataset

In [5]:
def explore_data(dataset, start, end, rows_and_columns=False):
    
    """
    Displays a specified slice of the dataset and optionally prints the dataset's dimensions.
    
        Parameters:
            dataset (list of lists): The dataset to be explored, where each inner list represents a row.
            start (int): Starting index of the slice.
            end (int): Ending index of the slice (non-inclusive).
            rows_and_columns (bool, optional): If True, prints the number of rows and columns in the dataset.

        Returns:
            None
            
    """
        
    dataset_slice = dataset[start:end] 
    
    for row in dataset_slice:
        print(row)
        print('\n') 

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [6]:
explore_data(moma, 0, 5, True)

['Title', 'Artist', 'Nationality', 'BeginDate', 'EndDate', 'Gender', 'Date', 'Department']


['Dress MacLeod from Tartan Sets', 'Sarah Charlesworth', '(American)', '(1947)', '(2013)', '(Female)', '1986', 'Prints & Illustrated Books']


['Duplicate of plate from folio 11 verso (supplementary suite, plate 4) from ARDICIA', 'Pablo Palazuelo', '(Spanish)', '(1916)', '(2007)', '(Male)', '1978', 'Prints & Illustrated Books']


['Tailpiece (page 55) from SAGESSE', 'Maurice Denis', '(French)', '(1870)', '(1943)', '(Male)', '1889-1911', 'Prints & Illustrated Books']


['Headpiece (page 129) from LIVRET DE FOLASTRIES, À JANOT PARISIEN', 'Aristide Maillol', '(French)', '(1861)', '(1944)', '(Male)', '1927-1940', 'Prints & Illustrated Books']


Number of rows: 16726
Number of columns: 8


As we can see the dataset consists of 16726 rows. The first one is header and each of the rest contains piece of information about one piece of art.


## Cleaning the nationality and gender columns

We can see that columns: nationality, gender have parentheses (()) at the beginning and the end of the values. So I will get rid of them. 

In [7]:
for row in moma_data:
    nationality = row[2]
    nationality_clean_open = nationality.replace('(', '')
    nationality_clean = nationality_clean_open.replace(')', '')
    row[2] = nationality_clean
    
    gender = row[5]
    gender_clean_open = gender.replace('(', '')
    gender_clean = gender_clean_open.replace(')', '')
    row[5] = gender_clean

In [8]:
explore_data(moma_data, 50, 55)

['PORTRAITS DE CHOPIN', 'Marc Chagall', 'French', '(1887)', '(1985)', 'Male', '1949', 'Prints & Illustrated Books']


['The Razorback Bunch (Etching III)', 'Robert Rauschenberg', 'American', '(1925)', '(2008)', 'Male', '1981', 'Prints & Illustrated Books']


['Identity Transfer from Program One: Aspen Projects', 'Dennis Oppenheim', 'American', '(1938)', '(2011)', 'Male', '1970', 'Media and Performance Art']


['Lipstick (Ascending) on Caterpillar Track', 'Claes Oldenburg', 'American', '(1929)', '', 'Male', '1969', 'Prints & Illustrated Books']


['"1 PERSON/1 PLACE" from the portfolio DETAILS FROM THE EXCAVATION OF WOOSTER STREET', 'Richard Nonas', 'American', '(1936)', '', 'Male', '1972', 'Prints & Illustrated Books']




There is also another problem with gender column. We can see that this column in our dataset can contain four different values: "", "Male", "Female", "male". 

In [9]:
for row in moma_data:
    gender = row[5]
    gender_clean = gender.title()
    row[5] =gender_clean
    if gender == "":
        gender = "Gender Unknown/Other"
        row[5] = gender
        
    nationality = row[2]
    nationality_clean = nationality.title()
    row[2] = nationality_clean
    if nationality == "":
        nationality = "Nationality Unknown"
        row[2] = nationality

## Cleaning BeginDate and EndDate columns

Next columns that need to be cleaned are: BeginDate which is artist's date of birth and EndDate which is artist's date of death. 
Values in both these columns are strings wraped by parenthesis. We want them to be integers with no parenthesis.

In [10]:
def clean_and_convert(date):
    if date != '':
        date = date.replace("(", "")
        date = date.replace(")", "")
        date = int(date)
    return date


In [11]:
for row in moma_data: 
    begin_date = row[3]
    end_date = row[4]
    row[3] = clean_and_convert(begin_date)
    row[4] = clean_and_convert(end_date)

In [12]:
explore_data(moma_data, 6, 10)

["Rue de l'Hôtel-de-Ville", 'Eugène Atget', 'French', 1857, 1927, 'Male', '1924', 'Photography']


['Los Angeles Airport', 'Garry Winogrand', 'American', 1928, 1984, 'Male', '1978-1983', 'Photography']


['Why Defy from Disasters of Peace', 'Diane Victor', 'South African', 1964, '', 'Female', '2001', 'Prints & Illustrated Books']


['STILL WATER AND FISH', 'David Brown Milne', 'Canadian', 1882, 1953, 'Male', '(1941)', 'Prints & Illustrated Books']




## Cleaning Date column 

The next column I will clean is the Date column. We can see that in that column there are values in different formats:  
* Some years are in parentheses.
* Some years have c. or C. before them, indicating that the year is approximate.
* Some have year ranges, indicated with a dash.
* Some have 's to indicate a decade
  
We want to get rid of all extra characters like: "(", ")", "c." etc. Then if the value is range, we want to calculate average date. 

In [13]:
bad_chars = ["(",")","c","C",".","s","'", " "]

def strip_characters(string):
    for char in bad_chars:
        string = string.replace(char,"")
    return string


In [14]:
def process_date(string):
    if "-" in string:
        splited_date = string.split("-")
        date_one = int(splited_date[0])
        date_two = int(splited_date[1])
        avg_date = (date_one + date_two) / 2
        date = round(avg_date)
    else:
        date = int(string)
        
    return date


In [15]:
for row in moma_data:
    date = row[6]
    date = strip_characters(date)
    date = process_date(date)
    row[6] = date

## Calcualting artist's age at which artist created piece of art

In [17]:
ages = []

for row in moma_data:
    birth_date = row[3]
    artwork_date = row[6]
    if type(birth_date) == int:
        age = artwork_date - birth_date 
    else: 
        age = 0 
    ages.append(age)
    
final_ages = [] 

for age in ages:
    if age > 20:
        final_age = age
    else: 
        final_age = "Unknown"
    final_ages.append(final_age)

Because there are many unique ages, we'll calculate only the decade during which the artist created each work. For instance, if we calculate that the artist was 24, we'll record that as the artist being in their "20s."

In [18]:
decades = []
for age in final_ages: 
    if age == "Unknown":
        decade = age 
    else: 
        decade = str(age)
        decade = decade[:-1]
        decade += "0s"
    decades.append(decade)

In [19]:
decade_frequency = {}
for decade in decades:
    if decade in decade_frequency:
        decade_frequency[decade] += 1
    else:
        decade_frequency[decade] = 1

## Creating an Artist Frequency Table

Next, I will create frequency table that will show how many pieces of art of each artist there are in our dataset.

In [20]:
artist_freq = {}

for row in moma: 
    artist = row[1]
    if artist not in artist_freq:
        artist_freq[artist] = 1
    else: 
        artist_freq[artist] += 1

In [23]:
def artist_summary(artist_name):
    artist_f = artist_freq[artist_name]
    output = f"There are {artist_f} artworks by {artist_name} in the dataset"
    print(output)


In [25]:
artist_summary("Henri Matisse")

There are 129 artworks by Henri Matisse in the dataset


## Creating Gender Frequency Table

Next, I will create frequency table that will show how many pieces of art of each gender there are in our dataset.

In [24]:
gender_freq = {}

for row in moma:
    gender = row[5]
    if gender in gender_freq:
        gender_freq[gender] += 1
    else:
        gender_freq[gender] = 1
        
for key, value in gender_freq.items():
    output = f"There are {value:,} artworks by {key} artists"
    print(output)

There are 1 artworks by Gender artists
There are 2,443 artworks by Female artists
There are 13,491 artworks by Male artists
There are 791 artworks by Gender Unknown/Other artists
